# Reddit Vaccine Myths

### Sentiment Analyze with Vader

In [ ]:
pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment import vader
import re
import nltk
import seaborn as sns
nltk.download('vader_lexicon')

In [ ]:
dados = pd.read_csv('../input/reddit-vaccine-myths/reddit_vm.csv')
dados.head()

### Cleaning Data

In [ ]:
dados.drop(['title','id','url','created'], axis=1, inplace=True)

In [ ]:
dados['timestamp'] = dados['timestamp'].str.split(' ')

In [ ]:
dados['date'] = dados['timestamp'].str.slice(0,1)
dados['hour'] = dados['timestamp'].str.slice(1)

In [ ]:
x = []
for n in dados['date']:
  x.append(str(n)[1:-1])

In [ ]:
dados['date'] = x

In [ ]:
y = []
for z in dados['hour']:
  y.append(str(z)[1:-1])

In [ ]:
dados['hour'] = y

In [ ]:
dados.drop('timestamp', axis=1, inplace=True)

In [ ]:
no_aps = []
for l in dados['date']:
  no_aps.append(l.replace("'", ""))

In [ ]:
dados['date'] = no_aps

In [ ]:
no_aps = []
for j in dados['hour']:
  no_aps.append(j.replace("'", ""))

In [ ]:
dados['hour'] = no_aps

In [ ]:
dados['date'] = pd.to_datetime(dados['date'], format="%Y-%m-%d")
dados['hour'] = pd.to_datetime(dados['hour'], format="%H:%M:%S")

In [ ]:
dados['year'] = dados['date'].map(lambda x: x.strftime("%Y"))
dados['month'] = dados['date'].map(lambda x: x.strftime("%m"))
dados['hour'] = dados['hour'].map(lambda x: x.strftime("%H"))

In [ ]:
dados.dropna(inplace=True)

In [ ]:
dados['body'] = dados['body'].str.lower()
dados['body'] = dados['body'].apply(lambda x: re.sub(r'[\@\:\_\.\;\#\"\'\$\%\&\/]', "", str(x)))

In [ ]:
dados['body'] = dados['body'].apply(lambda x: x.replace("https", ""))

In [ ]:
dados.head()

### Sentiment Analyze

In [ ]:
sentiment = vader.SentimentIntensityAnalyzer()

In [ ]:
sentiments = [sentiment.polarity_scores(i) for i in dados['body'].values]

In [ ]:
dados['Negative Score'] = [i['neg'] for i in sentiments]
dados['Positivo Score'] = [i['pos'] for i in sentiments]
dados['Neuter Score'] = [i['neu'] for i in sentiments]
dados['Compound Score'] = [i['compound'] for i in sentiments]

In [ ]:
dados.head()

In [ ]:
score = dados['Compound Score'].values

In [ ]:
t = []
for v in score:
  if v >= 0.05:
    t.append('Positive')
  elif v <= -0.05:
    t.append('Negative')
  else:
    t.append("Neuter")

dados['Sentimen'] = t

In [ ]:
dados.head()

In [ ]:
dados['Sentimen'].value_counts().plot(kind='bar')

In [ ]:
sns.histplot(dados['Compound Score'], bins=11)

In [ ]:
dados = pd.concat([dados, pd.get_dummies(dados['Sentimen'])], axis=1)
dados.head()

In [ ]:
dados2 = dados.groupby('year').mean()
dados2

In [ ]:
ax, fig = plt.subplots(figsize=(12,6))
plt.plot(dados2.index, dados2['Negative Score'], label='Negative')
plt.plot(dados2.index, dados2['Neuter Score'], label='Neuter')
plt.plot(dados2.index, dados2['Positivo Score'], label='Positive')
fig.spines['right'].set_visible(False)
fig.spines['top'].set_visible(False)
plt.title('Mean Composition by Year', fontsize=12, color='grey', style='oblique', fontweight='bold')
plt.xticks(color='grey', style='oblique', fontweight='bold')
plt.yticks(color='grey', style='oblique', fontweight='bold')
plt.xlabel("Year", fontsize=15, color='grey', style='oblique', fontweight='bold')
plt.ylabel("Composition", fontsize=15, color='grey', style='oblique', fontweight='bold')
plt.legend(fontsize=12)
plt.show()

In [ ]:
dados3 = dados.groupby('hour').mean()
dados3.head()

In [ ]:
ax, fig = plt.subplots(figsize=(12,6))
plt.plot(dados3.index, dados3['Negative Score'], label='Negative')
plt.plot(dados3.index, dados3['Neuter Score'], label='Neuter')
plt.plot(dados3.index, dados3['Positivo Score'], label='Positive')
fig.spines['right'].set_visible(False)
fig.spines['top'].set_visible(False)
plt.title('Mean Composition by Hour', fontsize=12, color='grey', style='oblique', fontweight='bold')
plt.xticks(color='grey', style='oblique', fontweight='bold')
plt.yticks(color='grey', style='oblique', fontweight='bold')
plt.xlabel("Year", fontsize=15, color='grey', style='oblique', fontweight='bold')
plt.ylabel("Composition", fontsize=15, color='grey', style='oblique', fontweight='bold')
plt.legend(fontsize=12)
plt.show()

In [ ]:
dados4 = dados.groupby('month').mean()
dados4.head()

In [ ]:
ax, fig = plt.subplots(figsize=(12,6))
plt.plot(dados4.index, dados4['Negative Score'], label='Negative')
plt.plot(dados4.index, dados4['Neuter Score'], label='Neuter')
plt.plot(dados4.index, dados4['Positivo Score'], label='Positive')
fig.spines['right'].set_visible(False)
fig.spines['top'].set_visible(False)
plt.title('Mean Composition by Month', fontsize=12, color='grey', style='oblique', fontweight='bold')
plt.xticks(color='grey', style='oblique', fontweight='bold')
plt.yticks(color='grey', style='oblique', fontweight='bold')
plt.xlabel("Year", fontsize=15, color='grey', style='oblique', fontweight='bold')
plt.ylabel("Composition", fontsize=15, color='grey', style='oblique', fontweight='bold')
plt.legend(fontsize=12)
plt.show()